In [2]:
from k8sclient.Components import (
    ServicePort,
    PodBuilder,
    ServiceBuilder,
    NetworkPolicyPort,
    NetworkPolicyEgressRule,
    NetworkPolicyIngressRule,
    NetworkPolicyBuilder,
)

from k8sclient.keywords import (
    list_ready_nodes,
    cleanup_pods,
    cleanup_services,
    is_pod_running,
    get_pod_ip,
    pod_exec,
    pod_exec_new,
    is_pod_running,
    switch_cluster,
    register_cluster,
    list_namespaced_services,
    list_all_services,
    list_namespaced_endpoints,
    get_service_ip,
    get_service,
    create_namespace,
    remove_namespace,
    list_namespaced_network_policy,
    list_network_policy_for_all_namespaces,
    delete_namespaced_network_policy_all,
    read_namespaced_network_policy,
    wait_for_pod_state,
    NOT_FOUND,
    SUCCEEDED,
    RUNNING,
    delete_pod,
    remove_pod,
    tail_pod_logs,
)


import random
import sys
import re
import argparse

import time
import requests
import json

In [2]:
# examples for list network policy
def list_network_policy(namespace):
    return list_namespaced_network_policy(namespace)
def list_all_network_policy():
    return list_network_policy_for_all_namespaces()

print "get networkPolicy of policy-demo"
nps = list_network_policy("policy-demo")
for np in nps:
    print np.metadata.name

print "get all networkPolicy"
nps = list_all_network_policy()
for np in nps:
    print np.metadata.name

get networkPolicy of policy-demo
get all networkPolicy
ingress-policy-0
ingress-policy-l0
ingress-policy-1
ingress-policy-l1
ingress-policy-2
ingress-policy-l2
ingress-policy-3
ingress-policy-l3


In [3]:
# examples for create egress and ingress rules

namespace_label = {"ns1":"ns1"}
pod_label = {"pod1":"pod1"}
port = NetworkPolicyPort(1234,protocol="TCP")

egress_rule = NetworkPolicyEgressRule()

egress_rule.add_policy_ipblock(cidr="172.10.0.0/16")
egress_rule.add_policy_namespace_selector(namespace_label=namespace_label)
egress_rule.add_policy_pod_selector(pod_label=pod_label)

egress_rule.add_policy_port(port)

egress_policy = NetworkPolicyBuilder(
    "xegress",
    "xujietest",
).add_egress_rule(
    egress_rule,
).add_policy_type(
    "Egress",
).add_pod_selector(
    "run1",
    "run1",
)

ingress_rule = NetworkPolicyIngressRule()
ingress_rule.add_policy_ipblock(cidr="172.20.0.0/16")
ingress_rule.add_policy_namespace_selector(namespace_label=namespace_label)
ingress_rule.add_policy_pod_selector(pod_label=pod_label)

ingress_rule.add_policy_port(port)

ingress_policy = NetworkPolicyBuilder(
    "xingress",
    "xujietest",
).add_ingress_rule(
    ingress_rule,
).add_policy_type(
    "Ingress",
).add_pod_selector(
    "run2",
    "run2",
)

egress_policy.deploy()
ingress_policy.deploy()

#egress_policy.un_deploy()
#ingress_policy.un_deploy()

In [4]:
# example for read network policy by given name and namespace

policy = read_namespaced_network_policy("xingress","xujietest")
policy_spec = policy.spec

print "policy_select:"
policy_select = policy_spec.pod_selector.match_labels
for k,v in policy_select.items():
    print (k + ":" + v)

print "policy_type:"
policy_type = policy_spec.policy_types
for i, type in enumerate(policy_type):
    print type

print "policy_ingress:"
policy_ingress = policy_spec.ingress
for i, ingress in enumerate(policy_ingress):
    for j, peer in enumerate(ingress._from):
        if peer.ip_block is not None:
            print "ip block:"
            print peer.ip_block.cidr
        if peer.namespace_selector is not None:
            print "namespace_selector:"
            namespace_selector = peer.namespace_selector.match_labels
            for k,v in namespace_selector.items():
                print (k + ":" + v)
        if peer.pod_selector is not None:
            print "pod_selector:"
            for k,v in peer.pod_selector.match_labels.items():
                print (k + ":" + v)
    for t, port in enumerate(ingress.ports):
        print "ports:"
        print "port:",port.port," protocol:",port.protocol

policy_select:
run2:run2
policy_type:
Ingress
policy_ingress:
ip block:
172.20.0.0/16
namespace_selector:
ns1:ns1
pod_selector:
pod1:pod1
ports:
port: 1234  protocol: TCP


In [6]:
# examples for update networkPolicy

namespace_label_1 = {"ns2":"ns2"}
pod_label_1 = {"pod2":"pod2"}
ingress_rule_1 = NetworkPolicyIngressRule()

ingress_rule_1.add_policy_namespace_selector(namespace_label=namespace_label_1)
ingress_rule_1.add_policy_pod_selector(pod_label=pod_label_1)

ingress_policy.add_ingress_rule(ingress_rule_1)

ingress_policy.update()

In [7]:
# examples for delete all network policies in given namespace

r = delete_namespaced_network_policy_all("xujietest")

{'api_version': 'networking.k8s.io/v1',
 'code': None,
 'details': None,
 'kind': 'NetworkPolicyList',
 'message': None,
 'metadata': {'_continue': None,
              'resource_version': '4261845',
              'self_link': '/apis/networking.k8s.io/v1/namespaces/xujietest/networkpolicies'},
 'reason': None,
 'status': None}

In [2]:
# examples for handle namespace

ns_label = {"ns1":"ns1"}
r = create_namespace("test1", ns_label)
#r = remove_namespace("test1")

In [20]:
# examples for handle pod and service
namespace = "test1"
pod_name = "test-pod1"
client_port = ServicePort("clientport", 8080, 8080)
def create_pod_service():
    inter_pod = PodBuilder(
        pod_name,
        namespace,
    ).set_node(
        "ubuntu-client1"
    ).add_container(
        pod_name,
        image="xujieasd/alphine-restcheck:0.5",
        # args=args,
        ports=[client_port],
        requests={'cpu': '0', "memory": '0'},
        limits = {'cpu': '1', "memory": '32Mi'}
    )
    inter_svc = ServiceBuilder("test-svc1", namespace).add_port(client_port)
    inter_svc.deploy(force=True)
    inter_pod.attache_service(inter_svc)
    inter_pod.deploy()

def cleanup():
    cleanup_pods(namespace=namespace)
    cleanup_services(namespace)

#create_pod_service()
cleanup()

In [21]:
# example to check bandwidth

image = "127.0.0.1:30100/library/alpine-iperf:0.1"
server_args = "iperf -f M -i 1 -m -s"
client_args = r"iperf -f M -t 10 -i 1 -c %s"
namespace = "k8sft"
nodes = sorted(list_ready_nodes())
nodes = [n for n in nodes if n.startswith("ubuntu-client")]
server_port = ServicePort("serverport", 5001, 5001)
global_server_name = "iperf-server"
glimit = {'cpu': '0', 'memory': '8Gi'}
grequest = {'cpu': '0', 'memory': '0'}
server_service = ServiceBuilder(global_server_name, namespace).add_port(server_port)
reports = {}
report_css = """<style>
table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
}
tr:nth-child(even) {background: #CCC}
tr:nth-child(odd) {background: #FFF}
</style>
"""
report_title = r"""<H1>Pod to Pod network throughput, single connection. (MBytes/sec)</H1>
Cell format: <b><i>[bw via ip]|[bw via service]</i></b>
<br>
CPU limit: <b>no limit</b>
<br>
Memory Limit: <b>8Gi</b>
<br>
Server cmd: <b>%s</b>
<br>
Client cmd: <b>%s</b>
<br>
<br>
""" % (server_args, client_args)

def add_report(src, dest, result):
    if src not in reports:
        reports[src] = {}
    if dest not in reports[src]:
        reports[src][dest] = result
    else:
        reports[src][dest] = "|".join([reports[src][dest], result])


def save_report():
    shorts = {}
    for n in nodes:
        shorts[n] = n.split(".")[-1]
    with open("report.html", "w") as f:
        f.write(report_css)
        f.write(report_title)
        f.write("<table><tr><th>From/To</th>\n")
        for n in nodes:
            f.write("<th>{}</th>\n".format(shorts[n]))
        f.write("</tr>\n")
        for n in nodes:
            if n not in reports:
                continue
            f.write("<tr><td>{}</td>\n".format(shorts[n]))
            for m in nodes:
                if m not in reports[n]:
                    f.write("<td>-</td>\n")
                else:
                    f.write("<td>{}</td>\n".format(reports[n][m]))
            f.write("</tr>\n")
        f.write("</table>\n")

def test(server_node, client_node):
    print client_node, "->", server_node
    server_pod_name = "server-" + "-".join(server_node.split("."))
    PodBuilder(
        server_pod_name, namespace
    ).add_container(
        name=server_pod_name + "-container",
        image=image,
        args=server_args,
        limits=glimit,
        requests=grequest,
        ports=[server_port]
    ).set_node(server_node).attache_service(server_service).deploy()
    # wait for server pod running
    wait_for_pod_state(namespace, server_pod_name, timeout=600, expect_status=RUNNING)
    time.sleep(3)
    # get server pod ip
    server_pod_ip = get_pod_ip(namespace, server_pod_name)
    run_client(client_node, server_node, server_pod_ip)
    run_client(client_node, server_node, global_server_name)
    delete_pod(namespace, server_pod_name)
    wait_for_pod_state(namespace, server_pod_name, timeout=240, expect_status=NOT_FOUND)

def run_server(server_node):
    server_pod_name = "server-" + "-".join(server_node.split("."))
    PodBuilder(
        server_pod_name, namespace
    ).add_container(
        name=server_pod_name + "-container",
        image=image,
        args=server_args,
        limits=glimit,
        requests=grequest,
        ports=[server_port]
    ).set_node(server_node).attache_service(server_service).deploy()
    try:
        # wait for server pod running
        wait_for_pod_state(namespace, server_pod_name, timeout=600, expect_status=RUNNING)
        time.sleep(5)
        # get server pod ip
        server_pod_ip = get_pod_ip(namespace, server_pod_name)
        for node in nodes:
            run_client(node, server_node, server_pod_ip)
            run_client(node, server_node, global_server_name)
    except:
        pass
    finally:
        remove_pod(namespace, server_pod_name)


def run_client(client_node, server_node, server_pod_ip):
    client_pod_name = "client-" + "-".join(client_node.split("."))
    pod = PodBuilder(
        client_pod_name, namespace
    ).add_container(
        name=client_pod_name + "-container",
        image=image,
        args=client_args % server_pod_ip,
        limits=glimit,
        requests=grequest,
    ).set_node(client_node)
    try:
        for i in range(2):
            pod.deploy()
            # wait for client complete
            wait_for_pod_state(namespace, client_pod_name, timeout=600, expect_status=SUCCEEDED)
            # parse client log to get tx speed.
            logs = tail_pod_logs(namespace, client_pod_name, lines=20).strip()
            print logs
            summary = logs.split("\n")[-1]
            m = re.match(r".*[^.\d]+([.\d]+) MBytes/sec", summary)
            if m:
                break
            remove_pod(namespace, client_pod_name)
        if m:
            print server_node, client_node, server_pod_ip, m.group(1)
            add_report(client_node, server_node, m.group(1))
        else:
            add_report(client_node, server_node, summary)
    except:
        pass
    finally:
        remove_pod(namespace, client_pod_name)

def cleanup():
    cleanup_pods(namespace=namespace)
    cleanup_services(namespace)


cleanup()
server_service.deploy(force=True)
# test all node pair
for node in nodes:
    run_server(node)

------------------------------------------------------------
Client connecting to 10.244.1.71, TCP port 5001
TCP window size: 0.04 MByte (default)
------------------------------------------------------------
[  3] local 10.244.1.72 port 45888 connected with 10.244.1.71 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0- 1.0 sec  6347 MBytes  6347 MBytes/sec
[  3]  1.0- 2.0 sec  6775 MBytes  6775 MBytes/sec
[  3]  2.0- 3.0 sec  6940 MBytes  6940 MBytes/sec
[  3]  3.0- 4.0 sec  6991 MBytes  6991 MBytes/sec
[  3]  4.0- 5.0 sec  6895 MBytes  6895 MBytes/sec
[  3]  5.0- 6.0 sec  6807 MBytes  6807 MBytes/sec
[  3]  6.0- 7.0 sec  6677 MBytes  6677 MBytes/sec
[  3]  7.0- 8.0 sec  7054 MBytes  7054 MBytes/sec
[  3]  8.0- 9.0 sec  6755 MBytes  6755 MBytes/sec
[  3]  9.0-10.0 sec  7001 MBytes  7001 MBytes/sec
[  3]  0.0-10.0 sec  68258 MBytes  6824 MBytes/sec
ubuntu-client1 ubuntu-client1 10.244.1.71 6824


------------------------------------------------------------
Client connecting to iperf-server, TCP port 5001
TCP window size: 0.04 MByte (default)
------------------------------------------------------------
[  3] local 10.244.1.73 port 39662 connected with 10.98.76.119 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0- 1.0 sec  5520 MBytes  5520 MBytes/sec
[  3]  1.0- 2.0 sec  5176 MBytes  5176 MBytes/sec
[  3]  2.0- 3.0 sec  5206 MBytes  5206 MBytes/sec
[  3]  3.0- 4.0 sec  5597 MBytes  5597 MBytes/sec
[  3]  4.0- 5.0 sec  6017 MBytes  6017 MBytes/sec
[  3]  5.0- 6.0 sec  6095 MBytes  6095 MBytes/sec
[  3]  6.0- 7.0 sec  6523 MBytes  6523 MBytes/sec
[  3]  7.0- 8.0 sec  6309 MBytes  6309 MBytes/sec
[  3]  8.0- 9.0 sec  6467 MBytes  6467 MBytes/sec
[  3]  9.0-10.0 sec  6563 MBytes  6563 MBytes/sec
[  3]  0.0-10.0 sec  59479 MBytes  5947 MBytes/sec
ubuntu-client1 ubuntu-client1 iperf-server 5947


------------------------------------------------------------
Client connecting to 10.244.1.71, TCP port 5001
TCP window size: 0.04 MByte (default)
------------------------------------------------------------
[  3] local 10.244.2.67 port 34414 connected with 10.244.1.71 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0- 1.0 sec   236 MBytes   236 MBytes/sec
[  3]  1.0- 2.0 sec   231 MBytes   231 MBytes/sec
[  3]  2.0- 3.0 sec   229 MBytes   229 MBytes/sec
[  3]  3.0- 4.0 sec   231 MBytes   231 MBytes/sec
[  3]  4.0- 5.0 sec   225 MBytes   225 MBytes/sec
[  3]  5.0- 6.0 sec   231 MBytes   231 MBytes/sec
[  3]  6.0- 7.0 sec   234 MBytes   234 MBytes/sec
[  3]  7.0- 8.0 sec   217 MBytes   217 MBytes/sec
[  3]  8.0- 9.0 sec   232 MBytes   232 MBytes/sec
[  3]  9.0-10.0 sec   225 MBytes   225 MBytes/sec
[  3]  0.0-10.0 sec  2291 MBytes   229 MBytes/sec
ubuntu-client1 ubuntu-client2 10.244.1.71 229


------------------------------------------------------------
Client connecting to iperf-server, TCP port 5001
TCP window size: 0.04 MByte (default)
------------------------------------------------------------
[  3] local 10.244.2.68 port 52482 connected with 10.98.76.119 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0- 1.0 sec   229 MBytes   229 MBytes/sec
[  3]  1.0- 2.0 sec   229 MBytes   229 MBytes/sec
[  3]  2.0- 3.0 sec   233 MBytes   233 MBytes/sec
[  3]  3.0- 4.0 sec   238 MBytes   238 MBytes/sec
[  3]  4.0- 5.0 sec   228 MBytes   228 MBytes/sec
[  3]  5.0- 6.0 sec   228 MBytes   228 MBytes/sec
[  3]  6.0- 7.0 sec   227 MBytes   227 MBytes/sec
[  3]  7.0- 8.0 sec   231 MBytes   231 MBytes/sec
[  3]  8.0- 9.0 sec   233 MBytes   233 MBytes/sec
[  3]  9.0-10.0 sec   232 MBytes   232 MBytes/sec
[  3]  0.0-10.0 sec  2307 MBytes   231 MBytes/sec
ubuntu-client1 ubuntu-client2 iperf-server 231


------------------------------------------------------------
Client connecting to 10.244.2.69, TCP port 5001
TCP window size: 0.04 MByte (default)
------------------------------------------------------------
[  3] local 10.244.1.74 port 54004 connected with 10.244.2.69 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0- 1.0 sec   238 MBytes   238 MBytes/sec
[  3]  1.0- 2.0 sec   233 MBytes   233 MBytes/sec
[  3]  2.0- 3.0 sec   232 MBytes   232 MBytes/sec
[  3]  3.0- 4.0 sec   227 MBytes   227 MBytes/sec
[  3]  4.0- 5.0 sec   229 MBytes   229 MBytes/sec
[  3]  5.0- 6.0 sec   227 MBytes   227 MBytes/sec
[  3]  6.0- 7.0 sec   227 MBytes   227 MBytes/sec
[  3]  7.0- 8.0 sec   222 MBytes   222 MBytes/sec
[  3]  8.0- 9.0 sec   229 MBytes   229 MBytes/sec
[  3]  9.0-10.0 sec   229 MBytes   229 MBytes/sec
[  3]  0.0-10.0 sec  2293 MBytes   229 MBytes/sec
ubuntu-client2 ubuntu-client1 10.244.2.69 229


------------------------------------------------------------
Client connecting to iperf-server, TCP port 5001
TCP window size: 0.04 MByte (default)
------------------------------------------------------------
[  3] local 10.244.1.75 port 52960 connected with 10.98.76.119 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0- 1.0 sec   227 MBytes   227 MBytes/sec
[  3]  1.0- 2.0 sec   227 MBytes   227 MBytes/sec
[  3]  2.0- 3.0 sec   225 MBytes   225 MBytes/sec
[  3]  3.0- 4.0 sec   222 MBytes   222 MBytes/sec
[  3]  4.0- 5.0 sec   229 MBytes   229 MBytes/sec
[  3]  5.0- 6.0 sec   232 MBytes   232 MBytes/sec
[  3]  6.0- 7.0 sec   235 MBytes   235 MBytes/sec
[  3]  7.0- 8.0 sec   221 MBytes   221 MBytes/sec
[  3]  8.0- 9.0 sec   227 MBytes   227 MBytes/sec
[  3]  9.0-10.0 sec   238 MBytes   238 MBytes/sec
[  3]  0.0-10.0 sec  2283 MBytes   228 MBytes/sec
ubuntu-client2 ubuntu-client1 iperf-server 228


------------------------------------------------------------
Client connecting to 10.244.2.69, TCP port 5001
TCP window size: 0.04 MByte (default)
------------------------------------------------------------
[  3] local 10.244.2.70 port 38738 connected with 10.244.2.69 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0- 1.0 sec  6379 MBytes  6379 MBytes/sec
[  3]  1.0- 2.0 sec  6907 MBytes  6907 MBytes/sec
[  3]  2.0- 3.0 sec  6795 MBytes  6795 MBytes/sec
[  3]  3.0- 4.0 sec  6809 MBytes  6809 MBytes/sec
[  3]  4.0- 5.0 sec  6979 MBytes  6979 MBytes/sec
[  3]  5.0- 6.0 sec  6647 MBytes  6647 MBytes/sec
[  3]  6.0- 7.0 sec  6935 MBytes  6935 MBytes/sec
[  3]  7.0- 8.0 sec  6670 MBytes  6670 MBytes/sec
[  3]  8.0- 9.0 sec  6999 MBytes  6999 MBytes/sec
[  3]  9.0-10.0 sec  6901 MBytes  6901 MBytes/sec
[  3]  0.0-10.0 sec  68086 MBytes  6803 MBytes/sec
ubuntu-client2 ubuntu-client2 10.244.2.69 6803


------------------------------------------------------------
Client connecting to iperf-server, TCP port 5001
TCP window size: 0.04 MByte (default)
------------------------------------------------------------
[  3] local 10.244.2.71 port 51214 connected with 10.98.76.119 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0- 1.0 sec  6095 MBytes  6095 MBytes/sec
[  3]  1.0- 2.0 sec  6560 MBytes  6560 MBytes/sec
[  3]  2.0- 3.0 sec  6495 MBytes  6495 MBytes/sec
[  3]  3.0- 4.0 sec  6628 MBytes  6628 MBytes/sec
[  3]  4.0- 5.0 sec  6411 MBytes  6411 MBytes/sec
[  3]  5.0- 6.0 sec  6576 MBytes  6576 MBytes/sec
[  3]  6.0- 7.0 sec  6355 MBytes  6355 MBytes/sec
[  3]  7.0- 8.0 sec  6441 MBytes  6441 MBytes/sec
[  3]  8.0- 9.0 sec  6580 MBytes  6580 MBytes/sec
[  3]  9.0-10.0 sec  6373 MBytes  6373 MBytes/sec
[  3]  0.0-10.0 sec  64525 MBytes  6451 MBytes/sec
ubuntu-client2 ubuntu-client2 iperf-server 6451


### below is examples for create network policy stress test environment
* will create N namespaces with labels
* for every nodes, create pods with labels for every namespace
* create svc for every pods
* for every namespaces, create 2 ingress networkpolicies
* one networkpolicy will create ingress rule which accept traffic from it's own namespace
* anther networkpolicy will create ingress rule which accept traffic from namespace[0] to namespace[N/2]

In [45]:
nodes = list_ready_nodes()
nodes = [n for n in nodes if n.startswith("ubuntu-client")]
image = "xujieasd/alphine-restcheck:0.5"
client_port = ServicePort("clientport", 8080, 8080)
counts = [0] * len(nodes)
readys = [True] * len(nodes)
dones = [False] * len(nodes)
node_marks = ["-".join(n.split(".")) for n in nodes]

NS_NUMBER = 4

In [46]:
def create_numbered_namespace():
    for i in range (0, NS_NUMBER):
        namespace_name = ("namespace-%d" % i)
        label = ("ns-%d" % i)
        namespace_label = {label:label}
        create_namespace(namespace_name, namespace_label)
  
def delete_numbered_namespace():
    for i in range (0, NS_NUMBER):
        namespace_name = ("namespace-%d" % i)
        remove_namespace(namespace_name)

def cleanup_all_pods():
    for i in range (0, NS_NUMBER):
        namespace_name = ("namespace-%d" % i)
        cleanup_pods(namespace_name)
        
def cleanup_all_services():
    for i in range (0, NS_NUMBER):
        namespace_name = ("namespace-%d" % i)
        cleanup_services(namespace_name)

def cleanup_all_network_policy():
    for i in range (0, NS_NUMBER):
        namespace_name = ("namespace-%d" % i)
        delete_namespaced_network_policy_all(namespace_name)

def cleanup():
    cleanup_all_network_policy()
    cleanup_all_services()
    cleanup_all_pods()
    delete_numbered_namespace()
 
def create_pods_services_peer_node():
    total = 0
    start = time.time()
    while not all(dones):
        # deploy ready nodes
        for i, n in enumerate(nodes):
            namespace_id = counts[i]
            namespace_name = ("namespace-%d" % namespace_id)
            if dones[i]:
                continue  
            pod_name = ("pod-%s-%d" % (node_marks[i], namespace_id))
            if readys[i]:
                # create a new pod
                inter_pod = PodBuilder(
                    pod_name,
                    namespace_name,
                ).set_node(
                    n
                ).add_container(
                    pod_name,
                    image=image,
                    # args=args,
                    ports=[client_port],
                    requests={'cpu': '0', "memory": '0'},
                    limits = {'cpu': '1', "memory": '32Mi'}
                )
                svc_name = ("svc-%s-%d" % (node_marks[i], namespace_id))
                inter_svc = ServiceBuilder(svc_name, namespace_name).add_port(client_port)
                inter_svc.deploy(force=True)
                inter_pod.attache_service(inter_svc)
                inter_pod.deploy()
                readys[i] = False
                # print "creating", pod_name
            else:
                # check for current pod running
                readys[i] = is_pod_running(namespace_name, pod_name)
                if readys[i]:
                    total += 1
                    counts[i] += 1
                if counts[i] >= NS_NUMBER:
                    print "It took %ds to deploy %d pod on %s" % (int(time.time()-start), NS_NUMBER, n)
                    # print n, "is done~!", "total", total
                    dones[i] = True
        time.sleep(3)
    print "it took", time.time() - start, "s"

def create_numbered_network_policy():
    for i in range (0, NS_NUMBER):
        namespace_name = ("namespace-%d" % i)
        policy_name = ("ingress-policy-l%d" % i)
        label = ("ns-%d" % i)
        namespace_label_local = {label:label}
        ingress_rule_l = NetworkPolicyIngressRule()
        ingress_rule_l.add_policy_namespace_selector(namespace_label=namespace_label_local)

        ingress_policy_l = NetworkPolicyBuilder(
            policy_name,
            namespace_name,
        ).add_ingress_rule(
            ingress_rule_l,
        ).add_policy_type(
            "Ingress",
        )
        ingress_policy_l.deploy()

        policy_name = ("ingress-policy-%d" % i)
        ingress_rule = NetworkPolicyIngressRule()
        half_num = NS_NUMBER/2
        for j in range (0, half_num):
            label = ("ns-%d" % j)
            namespace_label_local = {label:label}
            ingress_rule.add_policy_namespace_selector(namespace_label=namespace_label_local)

        ingress_policy = NetworkPolicyBuilder(
            policy_name,
            namespace_name,
        ).add_ingress_rule(
            ingress_rule,
        ).add_policy_type(
            "Ingress",
        )
        ingress_policy.deploy()
    
def policy_stress():
    create_numbered_namespace()
    create_pods_services_peer_node()
    create_numbered_network_policy()

policy_stress()
#cleanup()

It took 21s to deploy 4 pod on ubuntu-client1
It took 21s to deploy 4 pod on ubuntu-client2


it took 24.4726910591 s


### below will check if policy is correct, will check
* for all namespaces, check network traffic which are allowed to access all the pods in the given namespace
* for all namespaces. check network traffic which are not allowed to access all the pods in the given namespace 

In [57]:
SUCCESS_MARK = "CHECK_PASS"

def _get_pod_ip(namespace, pod):
    return get_pod_ip(namespace, pod)

def _check_service_connected(namespace, pod, service, port):
    s = time.time()
    o = pod_exec_new(namespace, pod, ["/opt/check2.sh", service, port])
    if o.find(SUCCESS_MARK) == -1:
        print "Fail to check %s:%s on %s, error message: [%s]. it took %s" % (service, port, pod, o, str(time.time() - s))
        return False
    return True

def _check_service_unconnected(namespace, pod, service, port):
    o = pod_exec_new(namespace, pod, ["/opt/check2.sh", service, port])
    if o.find(SUCCESS_MARK) != -1:
        print "Fail to check %s:%s on %s, error message: [%s]. expected unconnected " % (service, port, pod, o)
        return False
    return True

def check_service_connected(namespace, pod, service, port):
    # give it a retry
    return _check_service_connected(namespace, pod, service, port) or \
           _check_service_connected(namespace, pod, service, port)

def check_service_unconnected(namespace, pod, service, port):
    return _check_service_unconnected(namespace, pod, service, port)

def check_namespaced_pod_connection(namespace_id, namespace_set):
    
    err = []

    for nj, namespace_j in enumerate(namespace_set):
        namespace_name = ("namespace-%d" % namespace_j)
        for i, ni in enumerate(nodes):

            pod_name = ("pod-%s-%d" % (node_marks[i], namespace_j))
            print "namespace:%s pod_name: %s" % (namespace_name,pod_name)
            
            for k, nk in enumerate(nodes):
                if nk == ni and namespace_id == namespace_j:
                    continue
                svc_name = ("svc-%s-%d" % (node_marks[k], namespace_id))
                svc_namespace = ("namespace-%d" % namespace_id)
                svc = get_service(svc_namespace,svc_name)
                svc_ip = svc.spec.cluster_ip
                print "svc_name: %s, ip: %s, namespace: %s" % (svc_name, svc_ip, namespace_id)

                if not check_service_connected(namespace_name, pod_name, svc_ip, "8080"):
                    err_log = "Fail to connect %s:%s on %s." % (svc_name,"8080",pod_name)
                    err.append(err_log)
        
    return err

def check_namespace_pod_unconnection(namespace_id, namespace_set):
 
    err = []

    for nj, namespace_j in enumerate(namespace_set):
        namespace_name = ("namespace-%d" % namespace_j)
        for i, ni in enumerate(nodes):

            pod_name = ("pod-%s-%d" % (node_marks[i], namespace_j))
            print "namespace:%s pod_name: %s" % (namespace_name,pod_name)

            for k, nk in enumerate(nodes):
                if namespace_id == namespace_j:
                    continue
                svc_name = ("svc-%s-%d" % (node_marks[k], namespace_id))
                svc_namespace = ("namespace-%d" % namespace_id)
                svc = get_service(svc_namespace,svc_name)
                svc_ip = svc.spec.cluster_ip
                print "svc_name: %s, ip: %s, namespace: %s" % (svc_name, svc_ip, namespace_id)
    
                if not check_service_unconnected(namespace_name, pod_name, svc_ip, "8080"):
                    err_log = "unexpected connection %s:%s on %s." % (svc_name,"8080",pod_name)
                    err.append(err_log)
    return err

def check_all_pod_connection():
    err = []
    print "check connection"
    for i in range (0, NS_NUMBER):
        print "check namespace %d" % i
        namespace_set = []
        for j in range (0, NS_NUMBER/2):
            namespace_set.append(j)
        r = check_namespaced_pod_connection(i, namespace_set)
        if r:
            for i, rr in enumerate(r):
                err.append(rr)
    if err:
        print "connection issue found."
        for i, r in enumerate(err):
            print r
    else:
        print "no issue found"

def check_all_pod_unconnection():
    err = []
    print "check unconnection"
    for i in range (0, NS_NUMBER):
        namespace_set = []
        for j in range (NS_NUMBER/2, NS_NUMBER):
            namespace_set.append(j)
        r = check_namespace_pod_unconnection(i, namespace_set)
        if r:
            for i, rr in enumerate(r):
                err.append(rr)
    if err:
        print "connection issue found."
        for i, r in enumerate(err):
            print r
    else:
        print "no issue found"
#check_all_pod_connection()
check_all_pod_unconnection()

check unconnection
namespace:namespace-2 pod_name: pod-ubuntu-client1-2
svc_name: svc-ubuntu-client1-0, ip: 10.110.241.125, namespace: 0


svc_name: svc-ubuntu-client2-0, ip: 10.107.67.84, namespace: 0


namespace:namespace-2 pod_name: pod-ubuntu-client2-2
svc_name: svc-ubuntu-client1-0, ip: 10.110.241.125, namespace: 0


svc_name: svc-ubuntu-client2-0, ip: 10.107.67.84, namespace: 0


namespace:namespace-3 pod_name: pod-ubuntu-client1-3
svc_name: svc-ubuntu-client1-0, ip: 10.110.241.125, namespace: 0


...

namespace:namespace-2 pod_name: pod-ubuntu-client1-2
svc_name: svc-ubuntu-client1-3, ip: 10.98.214.117, namespace: 3


svc_name: svc-ubuntu-client2-3, ip: 10.109.208.189, namespace: 3


namespace:namespace-2 pod_name: pod-ubuntu-client2-2
svc_name: svc-ubuntu-client1-3, ip: 10.98.214.117, namespace: 3


svc_name: svc-ubuntu-client2-3, ip: 10.109.208.189, namespace: 3


namespace:namespace-3 pod_name: pod-ubuntu-client1-3
namespace:namespace-3 pod_name: pod-ubuntu-client2-3
no issue found
